In [18]:
import requests
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn')
import plotly.express as px

In [19]:
res = []
def get_vacancy(vacancies, pages=10):
    for vacancy in vacancies:
        print(f'collecting {vacancy}')
        for page in tqdm(range(pages)):
            params = {
                'text': f'NAME:{vacancy}',
                'page': page,
                'per_page': 100
                }
            req = requests.get('https://api.hh.ru/vacancies', params).json()
            res.extend(req['items'])

vacancies = ['аналитик', 'данные', 'data', 'analyst']
get_vacancy(vacancies)

df = pd.DataFrame(res)
df.shape

collecting аналитик


  0%|          | 0/10 [00:00<?, ?it/s]

collecting данные


  0%|          | 0/10 [00:00<?, ?it/s]

collecting data


  0%|          | 0/10 [00:00<?, ?it/s]

collecting analyst


  0%|          | 0/10 [00:00<?, ?it/s]

(3011, 30)

In [20]:
columns = ['name', 'salary', 'schedule', 'area']
df = df[columns]
df.head()

,name,salary,schedule,area
0,Аналитик,"{'from': 55000, 'to': None, 'currency': 'RUR',...","{'id': 'fullDay', 'name': 'Полный день'}","{'id': '2', 'name': 'Санкт-Петербург', 'url': ..."
1,Аналитик,None,"{'id': 'fullDay', 'name': 'Полный день'}","{'id': '41', 'name': 'Калининград', 'url': 'ht..."
2,Аналитик,None,"{'id': 'remote', 'name': 'Удаленная работа'}","{'id': '26', 'name': 'Воронеж', 'url': 'https:..."
3,Аналитик (Junior),"{'from': 100000, 'to': None, 'currency': 'RUR'...","{'id': 'fullDay', 'name': 'Полный день'}","{'id': '1', 'name': 'Москва', 'url': 'https://..."
4,Мидл аналитик,"{'from': 150000, 'to': None, 'currency': 'RUR'...","{'id': 'fullDay', 'name': 'Полный день'}","{'id': '1', 'name': 'Москва', 'url': 'https://..."


In [21]:
df["city"] = (df["area"]
                     .apply(lambda x: x.get("name") 
                                      if isinstance(x, dict) 
                                      else np.nan))
df["schedule"] = (df["schedule"]
                     .apply(lambda x: x.get("id") 
                                      if isinstance(x, dict) 
                                      else np.nan))
df["salary_from"] = (df["salary"]
                     .apply(lambda x: x.get("from") 
                                      if isinstance(x, dict) 
                                      else np.nan))
df["salary_to"] = (df["salary"]
                     .apply(lambda x: x.get("to") 
                                      if isinstance(x, dict) 
                                      else np.nan))
df["salary_currency"] = (df["salary"]
                     .apply(lambda x: x.get("currency") 
                                      if isinstance(x, dict) 
                                      else np.nan))

In [22]:
df.head()

,name,salary,schedule,area,city,salary_from,salary_to,salary_currency
0,Аналитик,"{'from': 55000, 'to': None, 'currency': 'RUR',...",fullDay,"{'id': '2', 'name': 'Санкт-Петербург', 'url': ...",Санкт-Петербург,55000.0,NaN,RUR
1,Аналитик,None,fullDay,"{'id': '41', 'name': 'Калининград', 'url': 'ht...",Калининград,NaN,NaN,NaN
2,Аналитик,None,remote,"{'id': '26', 'name': 'Воронеж', 'url': 'https:...",Воронеж,NaN,NaN,NaN
3,Аналитик (Junior),"{'from': 100000, 'to': None, 'currency': 'RUR'...",fullDay,"{'id': '1', 'name': 'Москва', 'url': 'https://...",Москва,100000.0,NaN,RUR
4,Мидл аналитик,"{'from': 150000, 'to': None, 'currency': 'RUR'...",fullDay,"{'id': '1', 'name': 'Москва', 'url': 'https://...",Москва,150000.0,NaN,RUR


In [23]:
df.tail()

,name,salary,schedule,area,city,salary_from,salary_to,salary_currency
3006,Бизнес-аналитик,None,fullDay,"{'id': '2', 'name': 'Санкт-Петербург', 'url': ...",Санкт-Петербург,NaN,NaN,NaN
3007,Ведущий аналитик 1С,"{'from': 250000, 'to': 300000, 'currency': 'RU...",fullDay,"{'id': '1', 'name': 'Москва', 'url': 'https://...",Москва,250000.0,300000.0,RUR
3008,Аналитик отдела продаж,"{'from': 50000, 'to': 70000, 'currency': 'RUR'...",fullDay,"{'id': '3', 'name': 'Екатеринбург', 'url': 'ht...",Екатеринбург,50000.0,70000.0,RUR
3009,Business Analyst / Бизнес-аналитик,"{'from': 50000, 'to': 90000, 'currency': 'RUR'...",remote,"{'id': '88', 'name': 'Казань', 'url': 'https:/...",Казань,50000.0,90000.0,RUR
3010,Бизнес-аналитик,None,fullDay,"{'id': '88', 'name': 'Казань', 'url': 'https:/...",Казань,NaN,NaN,NaN


In [24]:
df['salary_currency'].value_counts()

RUR    678
USD     38
KZT     19
EUR     12
UZS      6
BYR      4
AZN      1
Name: salary_currency, dtype: int64

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3011 entries, 0 to 3010
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             3011 non-null   object 
 1   salary           758 non-null    object 
 2   schedule         3011 non-null   object 
 3   area             3011 non-null   object 
 4   city             3011 non-null   object 
 5   salary_from      635 non-null    float64
 6   salary_to        478 non-null    float64
 7   salary_currency  758 non-null    object 
dtypes: float64(2), object(6)
memory usage: 188.3+ KB


In [26]:
df = df[df['salary_currency'] == 'RUR'][['city', 'salary_from', 'salary_to', 'salary_currency']]

In [27]:
df

,city,salary_from,salary_to,salary_currency
0,Санкт-Петербург,55000.0,NaN,RUR
3,Москва,100000.0,NaN,RUR
4,Москва,150000.0,NaN,RUR
5,Санкт-Петербург,70000.0,110000.0,RUR
7,Москва,250000.0,350000.0,RUR
...,...,...,...,...
2986,Москва,150000.0,NaN,RUR
2992,Москва,150000.0,150000.0,RUR
3007,Москва,250000.0,300000.0,RUR
3008,Екатеринбург,50000.0,70000.0,RUR


In [28]:
df['salary_from'].min(), df['salary_from'].max()

(15000.0, 350000.0)

In [29]:
df['salary_to'].min(), df['salary_to'].max()

(160.0, 450000.0)

In [30]:
df[df['salary_to'] == 160]

,city,salary_from,salary_to,salary_currency
194,Москва,NaN,160.0,RUR
